In [5]:
# Install & Import Libraries
%pip install torch torchvision matplotlib pillow

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from PIL import Image
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.3 MB ? eta -:--:--
   ---------------------------------------- 1.0/241.3 MB 2.5 MB/s eta 0:01:36
   ---------------------------------------- 1.6/241.3 MB 2.7 MB/s eta 0:01:29
   ---------------------------------------- 2.6/241.3 MB 3.1 MB/s eta 0:01:16
    --------------------------------------- 3.1/241.3 MB 3.1 MB/s eta 0:01:17
    --------------------------------------- 4.2/241.3 MB 3.4 MB/s eta 0:01:11
    --------------------------------------- 5.0/241.3 MB 3.5 MB/s eta 0:01:09
    ---

In [7]:
# Data Preparation & Augmentation
transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

transform_val = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Assuming dataset is in folders: data/train & data/val
if not os.path.exists("data/train") or not os.path.exists("data/val"):
    raise FileNotFoundError("Please make sure 'data/train' and 'data/val' directories exist and contain images.")

train_data = datasets.ImageFolder(root="data/train", transform=transform_train)
val_data   = datasets.ImageFolder(root="data/val", transform=transform_val)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)

print("Classes:", train_data.classes)

FileNotFoundError: Please make sure 'data/train' and 'data/val' directories exist and contain images.

In [ ]:
# Define CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = CNNModel().to(device)

In [ ]:
# Loss & Optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training Loop
num_epochs = 10
train_acc, val_acc = [], []

for epoch in range(num_epochs):
    model.train()
    correct, total = 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    train_accuracy = 100 * correct / total
    train_acc.append(train_accuracy)

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(images)
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_accuracy = 100 * correct / total
    val_acc.append(val_accuracy)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%")

In [ ]:
# Plot Accuracy Graph
plt.plot(range(1, num_epochs+1), train_acc, label="Train Accuracy")
plt.plot(range(1, num_epochs+1), val_acc, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()

In [ ]:
# Save the Trained Model
torch.save(model.state_dict(), "cat_dog_cnn.pth")
print("Model saved successfully!")

In [ ]:
# Load Model
loaded_model = CNNModel().to(device)
loaded_model.load_state_dict(torch.load("cat_dog_cnn.pth", map_location=device))
loaded_model.eval()
print("Model loaded successfully!")

In [ ]:
# Predict on External Image
def predict_image(image_path, model):
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image)
        prediction = (output > 0.5).float().item()
    
    class_names = train_data.classes
    print(f"Prediction: {class_names[int(prediction)]}")
    return class_names[int(prediction)]

# Example
# test_image_path = "test_cat.jpg"
# predict_image(test_image_path, loaded_model)